In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from os import walk
import random
import pickle
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical

In [8]:
# dataset is available on kaggle named face mask data
# getting images
path = 'face mask'

training_data = []
testing_data = []

dict_data = {
    'with_mask': 0,
    'without_mask': 1
}

for i, (dirpath, dirnames, filenames) in enumerate(walk(path)):
  if i > 1 and i < 4:
    lst = dirpath.split('/')
    for im in filenames:
      temp = cv2.imread(dirpath + '/' + im)
      img = cv2.resize(temp, (224, 224))
      training_data.append([img, dict_data[lst[-1]]])
  if i > 4:
    lst = dirpath.split('/')
    for im in filenames:
      temp = cv2.imread(dirpath + '/' + im)
      img = cv2.resize(temp, (224, 224))
      testing_data.append([img, dict_data[lst[-1]]])



In [12]:
# saving list into binary format using pickle

with open('training_data.pickle', 'wb') as f:
  pickle.dump(training_data, f)

with open('testing_data.pickle', 'wb') as f:
  pickle.dump(testing_data, f)

In [3]:
# getting saved data into pickle

with open('training_data.pickle', 'rb') as f:
  training_data = pickle.load(f)

with open('testing_data.pickle', 'rb') as f:
  testing_data = pickle.load(f)

In [4]:
# mixing the data
random.shuffle(training_data)
random.shuffle(testing_data)

In [5]:
# splitting data into features and labels
train_x = []
train_y = []
test_x = []
test_y = []

for x, y in training_data:
  train_x.append(x)
  train_y.append(y)

for x, y in testing_data:
  test_x.append(x)
  test_y.append(y)

In [6]:
train_x = np.array(train_x)
train_y = np.array(train_y)
test_x = np.array(test_x)
test_y = np.array(test_y)

In [7]:
# normalizing data

train_x = train_x/255.0
test_x = test_x/255.0

train_y = to_categorical(train_y)
test_y = to_categorical(test_y)

In [ ]:
# data is ready

In [8]:
# defining the model
def build_model(hp):
  model = keras.models.Sequential([
                                   keras.layers.Conv2D(
                                       filters = hp.Int('conv_1_filter', min_value = 32, max_value = 128, step = 16),
                                       kernel_size = hp.Choice('conv_1_kernel', values = [3, 5]),
                                       activation = 'relu',
                                       input_shape = train_x.shape[1:],
                                   ),
                                   keras.layers.Conv2D(
                                       filters = hp.Int('conv_2_filter', min_value = 32, max_value = 128, step = 16),
                                       kernel_size = hp.Choice('conv_2_kernel', values = [3, 5]),
                                       activation = 'relu',
                                   ),
                                   keras.layers.MaxPooling2D((2, 2)),
                                   keras.layers.Flatten(),
                                   keras.layers.Dense(
                                       units = hp.Int('dense_1_layer', min_value = 32, max_value = 128, step = 16),
                                       activation = 'relu',
                                   ),
                                   keras.layers.Dense(2, activation = 'softmax')  

  ])

  model.compile(
      optimizer = keras.optimizers.Adam(hp.Float('learning_rate', min_value = 1e-4, max_value = 1e-2, sampling = 'LOG', default=1e-3)),
      loss='categorical_crossentropy',
      metrics=['accuracy']
  )
  
  return model

In [9]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [10]:
tuner_search = RandomSearch(
    build_model,
    objective = 'val_accuracy',
    max_trials = 4,
    directory = 'output',
    project_name = 'classify_mask'
)

In [11]:
# searching models with best performance
tuner_search.search(train_x, train_y, epochs = 3, batch_size = 32, validation_data = (test_x, test_y))

Epoch 1/3
94/94 [==============================] - 45s 479ms/step - loss: 0.6725 - accuracy: 0.9013 - val_loss: 0.1145 - val_accuracy: 0.9617
Epoch 2/3
94/94 [==============================] - 39s 420ms/step - loss: 0.1041 - accuracy: 0.9647 - val_loss: 0.1287 - val_accuracy: 0.9534
Epoch 3/3
94/94 [==============================] - 44s 468ms/step - loss: 0.0480 - accuracy: 0.9840 - val_loss: 0.0676 - val_accuracy: 0.9803


Epoch 1/3
94/94 [==============================] - 42s 448ms/step - loss: 41.4102 - accuracy: 0.5763 - val_loss: 0.6300 - val_accuracy: 0.6615
Epoch 2/3
94/94 [==============================] - 40s 423ms/step - loss: 0.5116 - accuracy: 0.7487 - val_loss: 0.3285 - val_accuracy: 0.8747
Epoch 3/3
94/94 [==============================] - 40s 427ms/step - loss: 0.1960 - accuracy: 0.9257 - val_loss: 0.1931 - val_accuracy: 0.9306


Epoch 1/3
94/94 [==============================] - 48s 512ms/step - loss: 0.3679 - accuracy: 0.8517 - val_loss: 0.1537 - val_accuracy: 0.9379
Epoch 2/3
94/94 [==============================] - 46s 490ms/step - loss: 0.1261 - accuracy: 0.9557 - val_loss: 0.1236 - val_accuracy: 0.9596
Epoch 3/3
94/94 [==============================] - 46s 485ms/step - loss: 0.0820 - accuracy: 0.9727 - val_loss: 0.0777 - val_accuracy: 0.9741


Epoch 1/3
94/94 [==============================] - 64s 676ms/step - loss: 4.9721 - accuracy: 0.6470 - val_loss: 0.6647 - val_accuracy: 0.6656
Epoch 2/3
94/94 [==============================] - 54s 578ms/step - loss: 0.6981 - accuracy: 0.5247 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 3/3
94/94 [==============================] - 54s 572ms/step - loss: 0.6939 - accuracy: 0.4953 - val_loss: 0.6932 - val_accuracy: 0.5000


INFO:tensorflow:Oracle triggered exit


In [12]:
# getting best model out of all
model = tuner_search.get_best_models(num_models=1)[0]

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 220, 220, 48)      3648      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 218, 218, 80)      34640     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 109, 109, 80)      0         
_________________________________________________________________
flatten (Flatten)            (None, 950480)            0         
_________________________________________________________________
dense (Dense)                (None, 64)                60830784  
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 130       
Total params: 60,869,202
Trainable params: 60,869,202
Non-trainable params: 0
____________________________________________

In [14]:
# training our model
model.fit(train_x, train_y, epochs=4, batch_size=32, validation_data = (test_x, test_y))

Epoch 1/4
94/94 [==============================] - 39s 420ms/step - loss: 0.0214 - accuracy: 0.9933 - val_loss: 0.0791 - val_accuracy: 0.9772
Epoch 2/4
94/94 [==============================] - 40s 422ms/step - loss: 0.0620 - accuracy: 0.9790 - val_loss: 0.1094 - val_accuracy: 0.9669
Epoch 3/4
94/94 [==============================] - 40s 421ms/step - loss: 0.0493 - accuracy: 0.9833 - val_loss: 0.0720 - val_accuracy: 0.9772
Epoch 4/4
94/94 [==============================] - 40s 421ms/step - loss: 0.0184 - accuracy: 0.9950 - val_loss: 0.0526 - val_accuracy: 0.9834


In [ ]:
y_pred = model.predict(test_x)